In [49]:
############################# initialize parameter ####################################
# for quick access, to get the location of each sentiment in the dictionary
dic = {'START':0,'B-positive':1,'I-positive':2,'B-neutral':3,'I-neutral':4,'B-negative':5,'I-negative':6,'O':7,'STOP':8}
l = ['START', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative', 'O', 'STOP']
# store emission parameters
""" data structure: tuple + dictionary
    tuple: sentiment e.g.: 'B-positive' is first dic in this tuple
    dictionary: key: words, value: # of times it appears with this sentiment
                e.g.: {'my': 12} means that 'my' word appears 12 times in the sentiment 'o'
"""
e_param = ({},{},{},{},{},{},{},{}) ## 1st dict empty (start no emission)
# store transition parameters
"""data structure: matrix 8x8
   row: y(i) (from START to 'O'), column: y(i-1) (from 'B-positive' to STOP)
   to record # of transsition from y(i) to y(i-1)
   e.g.: (START->O): matrix[0][6] ([dic][dic-1])
"""
t_param = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]
          ,[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
# count # of sentiment appears+1
"""data structure: vector [8] (from START to 'O')
      vector[i] = sum(t_param[i])+1
"""
count = [0,0,0,0,0,0,0,0]

## read and parse file
train_file = open('train_EN','r', encoding='UTF-8') ###################need arg[0]!!!!!!!!!!!!!!
yi='START'
for r in train_file:
    try:
        var, sentiment = r.split()
        position = dic[sentiment] ## position: 1~7
        # update e_param
        if (var in e_param[position]):
            e_param[position][var] += 1
        else:
            e_param[position][var] = 1
            
        # update t_param
        pre_position = dic[yi]
        t_param[pre_position][position] += 1
        yi = sentiment
    except:
        # meaning the end of a sentence: x->STOP
        pre_position = dic[yi]
        t_param[pre_position][8] += 1
        yi = 'START'
# get count(yi)+1
for i in range(0,8):
    temp_sum = 0
    for j in range(0,9):
        temp_sum = temp_sum+t_param[i][j]
    count[i] = temp_sum+1 
# print (t_param)
# print (e_param)  
# print (count) 
## [1176, 378, 234, 1163, 717, 120, 51, 17471]

## convert transision param to probablity
for i in range(0,8):
    for j in range(0,8):
        t_param[i][j] = 1.0*t_param[i][j]/count[i]
# print (t_param)
"""
############################### Part 2 ######################################
dev_file = open('dev_EN.in','r', encoding='UTF-8')   #################################arg[1]!!!!!!!!!!!!!!
for r in dev_file:
    r = r.strip()
    if (r==''):
        print()
        continue
    temp_list = []
    #y* = argmax(e(x|y))
    for j in range(0,8):
        temp_dic = e_param[j]
        if (r in temp_dic):
            temp_list.append(temp_dic[r]/count[j])
        else:
            temp_list.append(1/count[j])
    #print(temp_list)
    max_value = max(temp_list)
    max_index = temp_list.index(max_value)   ######################### what if there are 2 max index???
    print(r," ",l[max_index]) ################################# how should the output format be? what does NOTE mean?
    #####################calculate precisona and recell later... :D

"""
############################### Part 3 ######################################
import pprint
pp = pprint.PrettyPrinter(indent=5)
def sentimentScore(preScore, x):
    """inputs: preScore: list of (pre_score real_num, pre_parent int)
               x: current word
       output: list of max(score real_num, parent int) for all states, len=7
    """
    score = []
    for i in range(1,8): # i: 1~7
        temp_score = []
        #calculate emission first
        if (x in e_param[i]):
            b = 1.0*e_param[i][x]/count[i]
        else:
            b = 1.0/count[i]
        for j in range(1,8): # j:1-7
            #score = preScore*a*b
            j_score = preScore[j-1][0]*(t_param[j][i])*b # trans 1~7 -> 1-7
            temp_score.append(j_score)
        max_value = max(temp_score)
        max_index = temp_score.index(max_value) # index: 0-6
        score.append((max_value, max_index))
    return score


def viterbiAlgo(X):
    """input: X, words list
       output: Y, sentiment list
    """
    #initialization
    Y = []
    preScore = []
    # start -> 1
    x = X[0]
    for j in range (1,8):
        if (x in e_param[j]):
            b = 1.0*e_param[j][x]/count[j]
        else:
            b = 1.0/count[j]
        prob = t_param[0][j]*b
        preScore.append((prob,0)) # (prob, START)   
    scores = [preScore]
    
    #calculate path i=(1,...,n)
    for i in range(1, len(X)):  # 1 -> n-1
        score = sentimentScore(scores[i-1], X[i]) # a list of max(score: real, parent: int) for all 7 states 
        scores.append(score)
        
    #calculate score(n+1, STOP), and get max
    temp_score = []
    for j in range(1,8):
        #score = preScore*a
        t_score = scores[len(X)-1][j-1][0]*(t_param[j][8])
        temp_score.append(t_score)
    max_value = max(temp_score)
    max_index = temp_score.index(max_value)
    scores.append([(max_value, max_index)])
    pp.pprint(scores)
    
    #backtracking
    parent = 0 # only 1 entry in STOP 
    for i in range(len(scores)-1,0,-1): #index range from len(score)-1 until 1 to, not include START
        parent = scores[i][parent][1]
        Y.insert(0, l[parent+1]) # 1-7
    print(Y)
    return Y

import pprint
dev_file = open('dev.in','r', encoding='UTF-8')   #################################arg[1]!!!!!!!!!!!!!!
out_file = open('dev_EN.out','w')
X = []
for r in dev_file:
    r = r.strip()
    if (r==''):
        #end of a sequence
        Y = viterbiAlgo(X)
        for i in range(0,len(X)):
#             print(X[i],"  ",Y[i])
            out_file.write(''+X[i]+" "+Y[i]+'\n')
#         print()
        out_file.write('\n')
        X=[]
    else:
        X.append(r)
#####################calculate precisona and recell later... :D
#{'START':0,'B-positive':1,'I-positive':2,'B-neutral':3,'I-neutral':4,'B-negative':5,'I-negative':6,'O':7,'STOP':8}


[    [    (0.0001372242018500522, 0),
          (0.0, 0),
          (0.001386281081650201, 0),
          (0.0, 0),
          (0.00017715419501133787, 0),
          (0.0, 0),
          (0.0010781715238897345, 0)],
     [    (5.093694425343966e-08, 6),
          (2.6063476134862715e-07, 0),
          (4.749122960728546e-08, 6),
          (8.611567020496024e-07, 2),
          (4.7312584758483376e-08, 6),
          (1.1868173195204008e-06, 4),
          (1.5972514229994998e-07, 6)],
     [    (7.546026294455663e-12, 6),
          (3.0939549067975687e-10, 1),
          (2.0009396245686456e-11, 5),
          (3.316721949036476e-10, 3),
          (1.0008794770450981e-11, 3),
          (4.106634323600003e-09, 5),
          (2.2025531604293664e-08, 5)],
     [    (1.0405703087322669e-12, 6),
          (3.672785976730258e-13, 1),
          (7.761433541619006e-12, 6),
          (1.2774265660405538e-13, 3),
          (9.66529729072471e-13, 6),
          (1.4209807348096897e-11, 5),
          (2.82